In [1]:
import pandas as pd
from pyomo.environ import Set, SolverFactory, ConcreteModel, Constraint, Param, Objective, Var, Binary, minimize
from datetime import datetime, timedelta, time
import ast

In [2]:
shipments = pd.read_excel("01-OR-RS-homework-data-template.xlsx", sheet_name='df_shipments')
df = pd.read_excel("01-OR-RS-homework-data-template.xlsx", sheet_name='df_schedule')
capacity = pd.read_excel("01-OR-RS-homework-data-template.xlsx", sheet_name='df_capacity')

In [3]:
def convert_string_to_list(s):
    # Replace colons with quotes to make it a valid Python expression
    s = s.replace(", ", ",").replace("[","").replace("]","")
    return s

df['new_departure']=df['scheduled_depart_weekday_time_local'].apply(convert_string_to_list)

In [4]:
def parse_new_departure(s):
    tuples = s.split('),(')
    tuples = [t.replace('(', '').replace(')', '') for t in tuples]
    parsed_tuples = [tuple(t.split(',')) for t in tuples]
    return parsed_tuples

In [5]:
# Apply the function to the new_departure column
df['parsed_departures'] = df['new_departure'].apply(parse_new_departure)

# Explode the parsed_departures column
schedule = df.explode('parsed_departures')

# Create the day_of_week and departure_time columns
schedule[['day_of_week', 'departure_time']] = pd.DataFrame(schedule['parsed_departures'].tolist(), index=schedule.index)

# Convert the day_of_week to integer
schedule['day_of_week'] = schedule['day_of_week'].astype(int)

# Drop the parsed_departures column
schedule['parsed_departure_time'] = schedule['departure_time'].apply(lambda x: datetime.strptime(x, "%H:%M").time())

# Reset index if necessary
schedule.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
schedule.head(5)

,carrier_id,origin_region,destination_region,scheduled_depart_weekday_time_local,travel_hrs_with_timezone_offset,truck_type,rate_type,rate_break,new_departure,parsed_departures,day_of_week,departure_time,parsed_departure_time
0,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(1, 20:00)",1,20:00,20:00:00
1,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(2, 20:00)",2,20:00,20:00:00
2,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(3, 20:00)",3,20:00,20:00:00
3,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(4, 20:00)",4,20:00,20:00:00
4,1234,LAX,ORD,"[(1, 20:00), (2, 20:00), (3, 20:00), (4, 20:00...",48,1,FTL,5000,"(1,20:00),(2,20:00),(3,20:00),(4,20:00),(5,20:00)","(5, 20:00)",5,20:00,20:00:00


In [6]:
model = ConcreteModel()

In [7]:
model.SHIPMENTS = Set(initialize=shipments['shipment_id'].tolist())
model.LINEHAULS = Set(initialize=schedule.index.tolist())

SHIPMENT PARAMETERS

In [8]:
# Parameters
shipment_weight = dict(zip(shipments['shipment_id'], shipments['weight_kg']))
shipment_pallet = dict(zip(shipments['shipment_id'], shipments['pallet']))
shipment_ready_time = dict(zip(shipments['shipment_id'], (pd.to_datetime(shipments['ready_time_local']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')))
shipment_due_time = dict(zip(shipments['shipment_id'], (pd.to_datetime(shipments['due_time_local']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')))

In [13]:
# Function to parse rate breaks
def parse_rate_break(rate_break):
    if isinstance(rate_break, str):
        rate_break = eval(rate_break)
    if isinstance(rate_break, list):
        rate_break = rate_break[0]
    return rate_break

schedule['parsed_rate_break'] = schedule['rate_break'].apply(parse_rate_break)

# Convert departure time to datetime objects
schedule['parsed_departure_time'] = schedule['departure_time'].apply(lambda x: datetime.strptime(x, "%H:%M").time())

# Parameters in the model
model.weight = Param(model.SHIPMENTS, initialize=shipment_weight)
model.pallet = Param(model.SHIPMENTS, initialize=shipment_pallet)
model.ready_time = Param(model.SHIPMENTS, initialize=shipment_ready_time)
model.due_time = Param(model.SHIPMENTS, initialize=shipment_due_time)
model.travel_time = Param(model.LINEHAULS, initialize=dict(zip(schedule.index, schedule['travel_hrs_with_timezone_offset'])))
model.day_of_week = Param(model.LINEHAULS, initialize=dict(zip(schedule.index, schedule['day_of_week'])))
model.departure_time = Param(model.LINEHAULS, initialize=dict(zip(schedule.index, ((schedule['parsed_departure_time']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s'))))

# Variables
model.x = Var(model.SHIPMENTS, model.LINEHAULS, domain=Binary)

# Trade-off parameter (0 <= alpha <= 1)
alpha = 0.5  # You can adjust this value to balance between cost and OTP

    'pyomo.core.base.param.IndexedParam'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
    usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.param.IndexedParam'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
    usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.param.IndexedParam'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
    usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.param.IndexedParam'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
    usually indicative of a modelling err

TypeError: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'

In [14]:
schedule['parsed_departure_time']

0     20:00:00
1     20:00:00
2     20:00:00
3     20:00:00
4     20:00:00
5     20:00:00
6     20:00:00
7     20:00:00
8     20:00:00
9     20:00:00
10    12:00:00
11    12:00:00
12    12:00:00
13    12:00:00
14    12:00:00
15    12:00:00
16    12:00:00
17    12:00:00
18    12:00:00
19    12:00:00
20    14:00:00
21    14:00:00
22    14:00:00
23    14:00:00
24    14:00:00
25    14:00:00
26    14:00:00
27    14:00:00
28    14:00:00
29    14:00:00
30    14:00:00
31    14:00:00
32    14:00:00
33    14:00:00
34    14:00:00
35    14:00:00
36    14:00:00
37    14:00:00
38    14:00:00
39    14:00:00
40    20:00:00
41    20:00:00
42    20:00:00
Name: parsed_departure_time, dtype: object

In [ ]:
# Function to calculate LTL cost
def calculate_ltl_cost(weight, rate_break):
    applicable_rate = max(rate_break.keys())
    for threshold, rate in sorted(rate_break.items()):
        if weight <= threshold:
            applicable_rate = rate
            break
    return weight * applicable_rate

# Objective function: Minimize weighted sum of cost and OTP
def obj_rule(model):
    cost_term = sum(model.x[s, l] * (model.weight[s] / capacity.loc[0, 'max_weight_kgs']) * schedule['parsed_rate_break'][l]
                    for s in model.SHIPMENTS for l in model.LINEHAULS if schedule.loc[l, 'rate_type'] == 'FTL') + \
                sum(model.x[s, l] * calculate_ltl_cost(model.weight[s], schedule['parsed_rate_break'][l])
                    for s in model.SHIPMENTS for l in model.LINEHAULS if schedule.loc[l, 'rate_type'] == 'LTL')
    
    otp_term = sum(model.x[s, l] * (model.due_time[s] - (model.ready_time[s] + model.travel_time[l] * 3600))
                   for s in model.SHIPMENTS for l in model.LINEHAULS if (model.due_time[s] - (model.ready_time[s] + model.travel_time[l] * 3600)) >= 0)
    
    return (1 - alpha) * cost_term - alpha * otp_term

model.obj = Objective(rule=obj_rule, sense=minimize)

In [ ]:
# Constraints
def route_constraint(model, s):
    return sum(model.x[s, l] for l in model.LINEHAULS) == 1
model.route_con = Constraint(model.SHIPMENTS, rule=route_constraint)

def weight_constraint(model, l):
    return sum(model.weight[s] * model.x[s, l] for s in model.SHIPMENTS) <= capacity.loc[0, 'max_weight_kgs']
model.weight_con = Constraint(model.LINEHAULS, rule=weight_constraint)

def pallet_constraint(model, l):
    return sum(model.pallet[s] * model.x[s, l] for s in model.SHIPMENTS) <= capacity.loc[0, 'max_pallet_count']
model.pallet_con = Constraint(model.LINEHAULS, rule=pallet_constraint)

In [ ]:
# Due time constraint
def due_time_constraint(model, s):
    return model.due_time[s] >= sum(model.ready_time[s] + model.travel_time[l] * 3600 * model.x[s, l] for l in model.LINEHAULS)
model.due_time_con = Constraint(model.SHIPMENTS, rule=due_time_constraint)

# Departure time constraint
def departure_time_constraint(model, s, l):
    ready_time = model.ready_time[s]
    day_of_week = model.day_of_week[l]
    departure_time = model.departure_time[l]

    ready_time_day = (ready_time + day_of_week * 86400) % (7 * 86400)
    departure_datetime = ready_time_day + departure_time

    return departure_datetime >= ready_time
model.departure_time_con = Constraint(model.SHIPMENTS, model.LINEHAULS, rule=departure_time_constraint)
